## Basic Concepts of Reinforcement Learning

**Markov Decision Process**:

- Environment has a set of states $\mathcal{S}$.

- Agent is given a set of possible actions $\mathcal{A}$.

- Environment dynamics: transitions from state $s_t$ to a new state $s_{t+1}$ according to the transition probability $P\left(s_{t+1}\|s_t, a_t\right)$ after the agent takes action $$a_t$$.

- Reward function: $r(s,a) = \mathbb{E}[r_{t+1}\|s_t = s, a_t = a]$ provides scalar rewards to the agent at each time step.

- Trajectory of an agent: 
  $$
  \tau = (s_1, a_1, r_1, s_2, a_2, r_2, s_3, a_3, r_3, ...)
  $$

**What to do given a MDP**:

1.  Policy search: Find a policy $\pi: \mathcal{S} \to \mathcal{A}$ that outputs actions for each given state such that the cumulative reward along the trajectory is maximized.
2. Inverse RL: Given a set of optimal trajectories, infer the corresponding MDP.

**Notations**:

- Returns:
  - Return (cumulative reward) starting step $t$: $G_t = r_{t+1} + ... + r_T$
  - If $T = \infty$, we can use discounted return where $\gamma \in [0,1]$ is a discounted factor. Discounted reward is defined as $G_t = \sum_{k=0}^\infty \gamma^k r_{t+k+1} = r_{t+1} + \gamma G_{t+1}.$
- Value function of a state $s$: $$V_\pi(s) = \mathbb{E}[G_t \mid s_t = s] = \mathbb{E}_\pi\left[\sum_{k=0}^T \gamma^k r_{t+k+1}  \mid s_t = s \right]$$.
- Q-Value of the state-action pair $(s,a)$: $$Q_\pi (s,a) = \mathbb{E}_\pi \left[G_t  \mid S_t = s, a_t = a \right] = \mathbb{E}_\pi \left[\sum_{k=0}^T \gamma^k r_{t+k+1}  \mid s_t = s, a_t = a \right]$$.

**Bellman Equations**:

- Value function of a state $s$:
$$
V_\pi(s) = \sum_a \pi(a  \mid s) \sum_{s^\prime} p(s^\prime |s,a)[r(s,a) + \gamma \mathbb{E}_\pi [G_{t+1}  \mid s_{t+1} = s']] = \sum_a \pi(a | s)\sum_{s^\prime} p(s^\prime | s,a) [r(s,a) + \gamma V_\pi(s^\prime)]
$$

- Value fuction of a state-action pair $(s,a)$:
$$
Q_\pi(s,a) = r(s,a) + \gamma \sum_{s^\prime} p(s^\prime,a) \sum_{a^\prime} \pi(a^\prime| s^\prime)Q_\pi(s^\prime, a^\prime)
$$

  

**Optimal Policies**:

- Thus, we could define $\pi \geq \pi\'$ if and only if $V_\pi(s) \geq V_{\pi^\prime}(s)$, for all $s \in \mathcal{S}$.
- The optimal values of states are $V^\*(s) = \max_\pi V_\pi(s) = \max_a \sum_{s^\prime} p(s^\prime  \mid s,a)[r(s,a) + \gamma V^\*(s^\prime)]$.
- The optimal values of state-action pairs are $Q^\*(s, a) = \max_\pi Q_\pi(s,a) = \sum_{s^\prime} p(s^\prime  \mid s,a)[r(s,a) + \gamma \max_{a^\prime} Q^\*(s^\prime, a^\prime)]$
- Finally, an optimal policy is defined to be 

$$
\pi(a | s) = \delta (a = \arg \max_a Q^* (s, a))
$$

and optimal trajectories are trajectories sampled from an optimal policy.

## RL & Control as Inference in GM

This section introduces how RL and control can be seen through the lens of graphical models and inference in graphical models. The basic idea is to define a *distribution* over trajectories that are desired or optimal. A great resource for this is [Sergey Levine's tutorial](https://arxiv.org/pdf/1805.00909.pdf)<d-cite key="levine2018reinforcement"></d-cite> on the same.

### A Graphical Model for RL: the MDP as a PGM

Here we describe how a general MDP (Markov Decision Process) can be modeled as a probabilistic graphical model.

<figure id="mdp-pgm" class="l-body-outset">
  <div class="row">
    <div class="col one">
      <img src="images/mdp_unconditional.png" style="height:120%;" />
    </div>
    <div class="col one">
      <img src="images/mdp_conditional.png" style="height:120%;" />
    </div>
  </div>
  <figcaption>
    <strong>MDP as a Graphical Model.</strong>
    <i>Left</i>: Basic structure of GM of MDP. <i>Right</i>: GM conditioned on optimality variables (reward).
  </figcaption>
</figure>

Consider the graphical model on the left. The state and action at every time step is modeled as a random variable in the graph. The graph is a chain-structure, Markovian DAG. The initial state is sampled according to some distribution $s_1 \sim p_1(s)$. Every state in the next time step depends on the previous state and previous action. This is called the *dynamics* of the environment, or the transition function: $s_{t+1} \sim p(s_{t+1} \mid s_t, a_t)$.

However, note that the selection of actions is kept very simple -- it does not even depend on the state! Every action $a_t$ is sampled according to some fixed probability distribution. This is because, before specifying any reward function, we have a uniform prior over what actions should be taken. They are completely random for the time being.

$$
  \begin{aligned}
    &\text{Initial state    }&   &s_1 \sim p_1(s)\\
    &\text{Transition    }&      &s_{t+1} \sim p(s_{t+1} \mid s_t, a_t)\\
    &\text{Policy    }& &a_t \sim \pi(a_t \mid s_t)\\
    &\text{Reward    }& &r_t = r(s_t, a_t)
  \end{aligned}
$$

The *unconditioned* graphical models takes random actions, leading to sampling of random trajectories. In order to sample optimal trajectories based on a reward function, we need to somehow bake the reward function into the graphical model.

We do so by introducing *optimality variables*. The optimal variable $\mathcal{O}_t$ at timestep $t$ is a binary random variable which depends on $s_t$ and $a_t$. It is "true", or takes the value 1 with probability $p(\mathcal{O}_t = 1 \mid s_t, a_t) = \exp(r(s_t, a_t))$. The higher the reward, the more likely it is that the optimality variable will be true.

$$
  {\color{red}
  \begin{aligned}
    &\text{Optimality    }&   &p(\mathcal{O}_t = 1 \mid s_t, a_t) = \text{exp}(r(s_t, a_t))
  \end{aligned}
  }
$$

So basically the reward $r_t$ is modeled as the log-probability of the optimality variable $\mathcal{O}_t$ being true. We will see how this helps recover the reinforcement learning objective when we do inference on this graphical model, while conditioning on all optimality variables being true.

The optimality variables are added to the graphical model on the right; you can see that the graphical model closely resembles an HMM, where the transition function is the same as that of the HMM, and the reward is captured through emission probabilities of optimality variables.

- This models defines a probability distribution over trajectories. The advantage of this formulation is that it not only gives us optimal solution (trajectory), but also partially-optimal or almost-optimal solutions that are also associated with high rewards. Learning such stochastic behavior is helpful for smarter exploration and also from a transfer learning point of view.

- Such a probabilistic model of the MDP provides a unified framework to ask different kinds of queries:
  1. Infer optimal trajectories given rewards - $p(\tau \mid \mathcal{O}_{1:T})$, where $\tau = (s_1, a_1, ..., s_T, a_T)$ is a trajectory.
  2. Infer rewards and action priors given optimal trajectories (demonstrations). This is called *inverse RL*.
  3. Infer policy given rewards $p(a_t \mid s_t, \mathcal{O}_{1:T})$. This is more closer to standard RL.

### Distributions over optmial trajectories

In this section, we derive the distributions over trajectories $\tau$ conditioned on optimality variables $\mathcal{O}_t$ under the model defined in the previous section.

$$
\newcommand{\opts}{\mathcal{O}_{1:T}}
\newcommand{\opt}{\mathcal{O}}
\begin{aligned}
  p(\tau \mid \opts) &\propto p(\tau, \opts)\\
                     &= p(s_1) \prod_{t=1}^T p(a_t \mid s_t)~p(s_{t+1} \mid s_t, a_t)~p(\opt_t \mid s_t, a_t)\\
                     &= p(s_1) \prod_{t=1}^T p(s_{t+1} \mid s_t, a_t)~\exp(r(s_t, a_t) + \log p(a_t \mid s_t))\\
                     &= \Bigg[ p(s_1) \prod_{t=1}^T p(s_{t+1} \mid s_t, a_t) \Bigg] \exp\Bigg(\underbrace{r(s_t, a_t)}_{\text{reward}} + \log \underbrace{p(a_t \mid s_t)}_{\text{action prior}} \Bigg)
\end{aligned}
$$

The action prior $p(a_t \mid s_t)$ is usually taken to be uniform, although it doesn't have to be. For example, if we don't want our agent to bump into walls, we can incorporate an appropriate action prior that prevents collisions into walls.

### Inferring reward and prior that generate trajectory

To connect this formulation to more of what we have seen in PGMs, specifically CRFs<d-cite key="lafferty2001conditional"></d-cite>, we consider the setting of *inverse RL* where given optimal trajectories executed by an optimal agent, we need to infer the best reward and priors that generate this trajectory.

The rewards and action priors are parametrized - they are linear functions of $\phi$ (and $\theta$) with features of the state (and action).

$$
\newcommand{\opts}{\mathcal{O}_{1:T}}
\newcommand{\opt}{\mathcal{O}}
\begin{aligned}
  p(\tau \mid \opts) &\propto \Bigg[ p(s_1) \prod_{t=1}^T p(s_{t+1} \mid s_t, a_t) \Bigg] \exp\Bigg( r_{\textcolor{red}{\phi}}(s_t, a_t) + \log p_{\textcolor{red}{\theta}}(a_t \mid s_t) \Bigg)\\
  &= \Bigg[ p(s_1) \prod_{t=1}^T p(s_{t+1} \mid s_t, a_t) \Bigg] \exp\Bigg(\textcolor{red}{\phi}^T \textbf{f}_r(s_t, a_t) + \textcolor{red}{\theta}^T \textbf{f}_p(s_t, a_t) \Bigg)
\end{aligned}
$$

The above formula looks like the featurized CRF<d-cite key="lafferty2001conditional"></d-cite>, where the parameters can be learned using maximum likelihood estimation.

### Optimal policy and planning via inference

Planning is taking actions in a state that provide the best possible future outcome. In this model, the optimal policy is defined as $p(a_t \mid s_t, \mathcal{O}_{t:T})$, i.e., the action to take given the current state, and conditioned on being optimal in all future time steps.

This reminds us of the HMM; we remarked above how the graphical model looks like an HMM. And we want to condition on emitted optimality variables in all future steps. Indeed analysis in this section will be similar to HMM, where we compute backward messages. The backward message $\beta_t$ is defined as

$$
\begin{aligned}
\textcolor{blue}{\beta_t(s_t, a_t)} &:= p(\mathcal{O}_{t:T} \mid s_t, a_t)\\
\textcolor{red}{\beta_t(s_t)} &:= p(\mathcal{O}_{t:T} \mid s_t)
\end{aligned}
$$

The recursion for backward messages can be obtained by expanding out $\beta_t(s_t, a_t)$ and $\beta_t(s_t)$.

$$
\begin{aligned}
  \textcolor{red}{\beta_t(s_t, a_t)} &= p(\mathcal{O}_{t:T} \mid s_t, a_t)\\
  &= \int_\mathcal{S} p(\mathcal{O}_{t:T}, s_{t+1} \mid s_t, a_t)~ds_{t+1}\\
  &= \int_\mathcal{S} \underbrace{p(\mathcal{O}_{t:T} \mid s_{t+1})}_{\textcolor{blue}{\beta_{t+1}(s_{t+1})}}~p(s_{t+1} \mid s_t, a_t)~p(\mathcal{O}_t \mid s_t, a_t)~ds_{t+1}\\
  &= p(\mathcal{O}_t \mid s_t, a_t)~\mathbb{E}_{s_{t+1} \sim p(s_{t+1} \mid s_t, a_t)} \Big[ \textcolor{blue}{\beta_{t+1}(s_t+1)} \Big]\\
  \textcolor{blue}{\beta_t(s_t)} &= p(\mathcal{O}_{t:T} \mid s_t)\\
  &= \int_{\mathcal{A}} p(\mathcal{O}_{t:T} \mid s_t, a_t)~p(a_t \mid s_t)~da_t\\
  &= \mathbb{E}_{a_t \sim p(a_t \mid s_t)} \Big[ \textcolor{red}{\beta_t(s_t, a_t)} \Big]
\end{aligned}
$$

So the algorithm to compute backward messages is

$$
\begin{aligned}
\text{for } t &= T - 1 \text{ to } 1:\\
& \textcolor{red}{\beta_t(s_t, a_t)} = p(\mathcal{O}_t \mid s_t, a_t)~\mathbb{E}_{s_{t+1} \sim p(s_{t+1} \mid s_t, a_t)} \Big[ \textcolor{blue}{\beta_{t+1}(s_t+1)}\Big]\\
& \textcolor{blue}{\beta_t(s_t)} = \mathbb{E}_{a_t \sim p(a_t \mid s_t)} \Big[ \textcolor{red}{\beta_t(s_t, a_t)} \Big]
\end{aligned}
$$

### Relating backward messages to RL

If we define

$$
\begin{aligned}
&V_t(s_t) = \log \textcolor{blue}{\beta_t(s_t)}\\
&Q_t(s_t, a_t) = \log \textcolor{red}{\beta_t(s_t, a_t)}
\end{aligned}
$$

and if we assume uniform action prior, then from the previous equations, we have

$$
\begin{aligned}
V_t(s_t) &= \log \int \exp(Q_t(s_t, a_t))~da_t\\
Q_t(s_t, a_t) &= r(s_t, a_t) + \log \mathbb{E}_{s_{t+1} \sim p(s_{t+1} \mid s_t, a_t)} \Big[ \exp(V_{t+1}(s_{t+1})) \Big]
\end{aligned}
$$

This is similar to the relationship between the Q-function and V-function in standard RL. Execpt that in the first equation, a log-integral-exponential approximates a max (also called "soft-max"). And in the second equation, the log-expectation-exponential is similar to a log-integral-exponential which is similar to a soft-max.

Let us carefully consider the recursive equation for $Q_t(s_t, a_t)$. For the case of deterministic dynamics, the expectation over $s_{t+1}$ reduces to a single evaluation over $s_{t+1}$, and we get $Q_t(s_t, a_t) = r(s_t, a_t) + V_{t+1}(s_{t+1})$. This is the same equation for deterministic dynamics of standard RL.

However, in general for stochastic dynamics, we end up taking the approximate max of $V(s_{t+1})$ over all $s_{t+1}$. That is, we end up being too "optimistic" about the transition. This is problematic because this results in learning risk-seeking behaviors where actions are chosen that may rarely lead to some states that are associated with high reward. We will see in the next section how such optimistic behavior can be addressed.

### Optimal Policy

In the homework, we will show that the optimal policy is given by

$$
\pi(a_t \mid s_t) = \frac{\beta_t(s_t, a_t)}{\beta_t(s_t)} = \exp \Big( Q_t(s_t, a_t) - V(s_t) \Big) = \exp(A_t(s_t, a_t)))
$$

where $A_t(s_t, a_t)$ is known as the "advantage" function, since it denotes the advantage of taking action $a_t$ with respect to the "average" action for that state. There are two advantages of this form -

- It has a natural interpretation, that better actions (which have high advantage) are more probable.
- It facilitates random tie-breaking: if two actions have the same advantage - since the policy is stochastic, it samples them equally likely. Hence, this provides an automatic tie-breaking mechanism.
- We can add a temperature parameter inside the exponent, which can either focus on sharp peaks (converging to a deterministic policy), or sample more uniformly (converging to a completely randomly policy).

***

## Control via Variational Inference

Given this framework of inference for reinforcement learning with graphical models, one can ask the question about the relationship between the objective function the inference-basd policy is trying to optimize. In other words, what objective does inference optimize? We take a look at the KL divergence between trajectory distributions.

In the case for deterministic dynamics, we can express the probability of an optimal trajectory $\tau$ as

$$
p(\tau) \propto \left[ p(\mathbf{s}_1) \prod_{t=1}^T p(\mathbf{s}_{t+1} | \mathbf{s}_t \mathbf{a}_t) \right] \exp \left( \sum_{t=1}^T r(\mathbf{s}_t, \mathbf{a}_t) \right).
$$

On the other hand, the inference-based policy produces the following probability of the trajectory $\tau$:

$$
\hat{p}(\tau) \propto \mathbf{1}[p(\tau) \neq 0] \prod_{t=1}^T \pi(\mathbf{a}_t | \mathbf{s}_t).
$$

Computing the KL divergence between these two distributions, we have

<d-math block>
\begin{aligned}
-D_{KL}(\hat{p}(\tau) || p(\tau)) &= \mathbb{E}_{\tau \sim \hat{p}(\tau)} \left[ \sum_{t=1}^T r(\mathbf{s}_t, \mathbf{a}_t) - \log \pi(\mathbf{a}_t | \mathbf{s}_t) \right] \\
&= \sum_{t=1}^T \mathbb{E}_{(\mathbf{s}_t, \mathbf{a}_t) \sim \hat{p}(\mathbf{s}_t, \mathbf{a}_t)}[r(\mathbf{s}_t, \mathbf{a}_t))] + \mathbb{E}_{\mathbf{s}_t \sim \hat{p}(\mathbf{s}_t) }[H(\pi(\mathbf{a}_t | \mathbf{s}_t))]
\end{aligned}
</d-math>
where $H$ is the entropy. So the objective is to maximize this quantity -- the first term being the expected return (which is a standard RL objective), and the second term being the entropy of the policy, which helps promote stochasticity for exploration.

Now that we have seen the objective for the deterministic dynamics, let us think about the stochastic case as well. For the deterministic case, we have
$$
Q(\mathbf{s}_t , \mathbf{a}_t) = r(\mathbf{s}_t, \mathbf{a}_t) + V(\mathbf{s}_{t+1})
$$
while for stochastic dynamics we will have,
$$
Q(\mathbf{s}_t , \mathbf{a}_t) = r(\mathbf{s}_t, \mathbf{a}_t) + \log \mathbb{E}_{\mathbf{s}_{t+1} \sim p(\mathbf{s}_{t+1} | \mathbf{s}_t, \mathbf{a}_t)} [\exp(V(\mathbf{s}_{t+1}))]
$$
where the value function for the second term is now an expectation over all possible next states according to the transition distribution $p$. The second term however is not desirable because this make the Q function to be "optimistic" -- if any of the future states have a high reward regardless of the intermediary states that lead to there, the exponential term will favor that high-reward-state only and disregard all other states. This means that the agent may present risk-seeking behavior as long as it has some non-zero probability of obtaining a high reward at the end. In the end of optimizing this objective, we will not be sure if the policy learned was indeed good, or we just got lucky with the stochastic dynamics.

Another problem for the stochastic dynamics is that the optimized trajectory distribution becomes
$$
\hat{p}(\tau) = p(\mathbf{s}_1 | \mathcal{O}_{1:T}) \prod_{t=1}^T p(\mathbf{s}_{t+1} | \mathbf{s}_t , \mathbf{a}_t, \mathcal{O}_{1:T}) p(\mathbf{a}_t | \mathbf{s}_t \mathcal{O}_{1:T})
$$
which means that now the transition probability also depends on the optimality. What this means is that the agent can control both the actions and the dynamics of the system to create optimal trajectories, which is not desirable. We want the transition probability to remain the same regardless of the optimality.

To address these two issues, we use variational inference. Basically the goal is to find a variational distribution

$$ q(\mathbf{s}_{1:T}, \mathbf{a}_{1:T})$$

which approximates

$$p(\mathbf{s}_{1:T}, \mathbf{a}_{1:T} | \mathcal{O}_{1:T})$$

while keeping the transition probabilities independent from the optimality. So the graphical model now looks slightly different as the figure below.

<figure id="vi" class="l-body-outset">
  <div class="row">
    <div class="col two">
      <img src="images/vi.PNG" />
    </div>
  <figcaption>
    <strong>Variational Inference for Stochastic Dynamics</strong>
    Above is the original formulation of the trajectory distribution based on optimality, and using variational inference, we try to find a variational distribution(bottom) that approximates the original distribution without the optimality variable.
  </figcaption>
  </div>
</figure>

Following the figure, we let the probability of a trajectory produce by the policy as
$$
q(\mathbf{s}_{1:T}, \mathbf{a}_{1:T}) = p(\mathbf{s}_1) \prod_{t=1}^T p(\mathbf{s}_{t+1} | \mathbf{s}_t , \mathbf{a}_t) q(\mathbf{a}_t | \mathbf{s}_t).
$$
where we introduce a variational policy term $q$ with the given stochastic dynamics.

Now using a standard approach in variational inference, we have the following ELBO:

<d-math block>
\begin{aligned}
\log p(\mathcal{O}_{1:T}) &= \log \int \int p(\mathcal{O}_{1:T}, \mathbf{s}_{1:T}, \mathbf{a}_{1:T}) d\mathbf{s}_{1:T} d\mathbf{a}_{1:T} \\
&=  \log \int \int p(\mathcal{O}_{1:T}, \mathbf{s}_{1:T}, \mathbf{a}_{1:T}) \frac{q(\mathbf{s}_{1:T}, \mathbf{a}_{1:T})}{q(\mathbf{s}_{1:T}, \mathbf{a}_{1:T})} d\mathbf{s}_{1:T} d\mathbf{a}_{1:T} \\
&= \log \mathbb{E}_{(\mathbf{s}_{1:T}, \mathbf{a}_{1:T}) \sim q(\mathbf{s}_{1:T}, \mathbf{a}_{1:T})}\left[ \frac{p(\mathcal{O}_{1:T}, \mathbf{s}_{1:T}, \mathbf{a}_{1:T})}{q(\mathbf{s}_{1:T}, \mathbf{a}_{1:T})} \right] \\
&\geq \mathbb{E}_{(\mathbf{s}_{1:T}, \mathbf{a}_{1:T}) \sim q(\mathbf{s}_{1:T}, \mathbf{a}_{1:T})} [\log p(\mathcal{O}_{1:T}, \mathbf{s}_{1:T}, \mathbf{a}_{1:T}) - \log q(\mathbf{s}_{1:T}, \mathbf{a}_{1:T})] \\
&= \mathbb{E}_{\tau \sim q} \left[\sum_{t=1}^T r(\mathbf{s}_t, \mathbf{a}_t) - \log q(\mathbf{a}_t | \mathbf{s}_t ) \right] \\
&= \sum_{t=1}^T \mathbb{E}_{(\mathbf{s}_t, \mathbf{a}_t) \sim q} [r(\mathbf{s}_t, \mathbf{a}_t)] + H(q(\mathbf{a}_t | \mathbf{s}_t ))
\end{aligned}
</d-math>

where the inequality follows from Jensen's inequality. Now notice that the objective now is composed of two components just like the deterministic case, but in terms of the variational  distribution. The first term is the expected return induced by the variational policy, and the second term is the entropy of the variational policy. Notice that this objective only allows the agent to modify the policy and not the dynamics, while keeping the overall form same as the deterministic case. So the stochastic case maximizes this ELBO in search for an optimal $q$.

Using the similar approach to the deterministic case, we obtain the following expressions for the Value function and the Q function,
$$
V_t(\mathbf{s}_t) = \log \int \exp(Q_t(\mathbf{s}_t, \mathbf{a}_t)) d\mathbf{a}_t
$$

$$
Q_t(\mathbf{s}_t, \mathbf{a}_t) = r(\mathbf{s}_t, \mathbf{s}_a) + \mathbb{E}[V_{t+1}(\mathbf{s}_{t+1})]
$$
along with the expression for the variational policy
$$
q(\mathbf{a}_t | \mathbf{s}_t) = \exp(Q(\mathbf{s}_t , \mathbf{a}_t) - V(\mathbf{s}_t)),
$$
but now with a guarantee that (1) the agent does not manipulate the dynamics of the system, and (2) the optimism introduced from previous framework is no longer an issue as the update on Q function does not involve exponent of values.

<!-- ## Equations

This theme supports rendering beautiful math in inline and display modes using [KaTeX](https://khan.github.io/KaTeX/) engine.
You just need to surround your math expression with `$`, like `$ E = mc^2 $`.
If you leave it inside a paragraph, it will produce an inline expression, just like $E = mc^2$.

To use display mode, again surround your expression with `$$` and place it as a separate paragraph.
Here is an example:
$$
\left( \sum_{k=1}^n a_k b_k \right)^2 \leq \left( \sum_{k=1}^n a_k^2 \right) \left( \sum_{k=1}^n b_k^2 \right)
$$
Alternatively and for more complex math environments, use `<d-math block>...</d-math>` tags.
Here is an example:

<d-math block>
\begin{aligned}
\left( \sum_{i=1}^n u_i v_i \right)^2 & \leq \left( \sum_{i=1}^n u_i^2 \right) \left( \sum_{i=1}^n v_i^2 \right) \\
\left| \int f(x) \overline{g(x)} dx \right|^2 & \leq  \int |f(x)|^2 dx \int |g(x)|^2 dx
\end{aligned}
</d-math>

Note that [KaTeX](https://khan.github.io/KaTeX/) is work in progress, so it does not support the full range of math expressions as, say, [MathJax](https://www.mathjax.org/).
Yet, it is [blazing fast](http://www.intmath.com/cg5/katex-mathjax-comparison.php).

***

## Figures

To add figures, use `<figure>...</figure>` tags.
Within the tags, define multiple rows of images using `<div class="row">...</div>`.
To add captions, use `<figcaption>...</figcaption>` tags.

Here is an example usage of a figure that consists of a row of images with a caption:

<figure id="example-figure" class="l-body-outset">
  <div class="row">
    <div class="col one">
      <img src="{{ 'assets/img/notes/template/1.jpg" />
    </div>
    <div class="col one">
      <img src="{{ 'assets/img/notes/template/2.jpg" />
    </div>
    <div class="col one">
      <img src="{{ 'assets/img/notes/template/3.jpg" />
    </div>
  </div>
  <figcaption>
    <strong>Figure caption title in bold.</strong>
    An example figure caption.
  </figcaption>
</figure>

Note that the figure uses `class="l-body-outset"` which lets it take more horizontal space.
For more on this, see layouts section below.
Also, the size of the images themselves is controlled by `class="one"`, `class="two"`, or `class="three"` which corresponds to 1/3, 2/3, 3/3 of the full horizontal space, respectively.

Here is the same example, but each image is captioned separately:
<figure id="example-figure" class="l-body-outset">
  <div class="row">
    <div class="col one">
      <img src="{{ 'assets/img/notes/template/1.jpg" />
      <figcaption>
        <strong>Figure caption title 1.</strong>
        Caption text for figure 1.
      </figcaption>
    </div>
    <div class="col one">
      <img src="{{ 'assets/img/notes/template/2.jpg" />
      <figcaption>
        <strong>Figure caption title 2.</strong>
        A very very very long caption text for figure 2 so that it is longer than the image itself.
      </figcaption>
    </div>
  </div>
</figure>

Here is an example that shows how the figures of different sizes are aligned:

<figure>
  <div class="row">
    <div class="col two">
      <img src="{{ 'assets/img/notes/template/4.jpg" />
    </div>
    <div class="col one">
      <img src="{{ 'assets/img/notes/template/2.jpg" />
      <figcaption>
        <strong>Subcaption.</strong>
        The content of the subcaption.
      </figcaption>
    </div>
  </div>
  <figcaption>
    <strong>The second row figure caption title.</strong>
    An example of a sencond row figure caption.
  </figcaption>
</figure>
***

## Citations

Citations are then used in the article body with the `<d-cite>` tag.
The key attribute is a reference to the id provided in the bibliography.
The key attribute can take multiple ids, separated by commas.

The citation is presented inline like this: <d-cite key="gregor2015draw"></d-cite> (a number that displays more information on hover).
If you have an appendix, a bibliography is automatically created and populated in it.

Distill chose a numerical inline citation style to improve readability of citation dense articles and because many of the benefits of longer citations are obviated by displaying more information on hover.
However, we consider it good style to mention author last names if you discuss something at length and it fits into the flow well — the authors are human and it’s nice for them to have the community associate them with their work.

***

## Footnotes

Just wrap the text you would like to show up in a footnote in a `<d-footnote>` tag.
The number of the footnote will be automatically generated.<d-footnote>This will become a hoverable footnote.</d-footnote>

***

## Code Blocks

Syntax highlighting is provided within `<d-code>` tags.
An example of inline code snippets: `<d-code language="html">let x = 10;</d-code>`.
For larger blocks of code, add a `block` attribute:

<d-code block language="javascript">
  var x = 25;
  function(x) {
    return x * x;
  }
</d-code>

***

## Layouts

The main text column is referred to as the body.
It is the assumed layout of any direct descendants of the `d-article` element.

<div class="fake-img l-body">
  <p>.l-body</p>
</div>

For images you want to display a little larger, try `.l-page`:

<div class="fake-img l-page">
  <p>.l-page</p>
</div>

All of these have an outset variant if you want to poke out from the body text a little bit.
For instance:

<div class="fake-img l-body-outset">
  <p>.l-body-outset</p>
</div>

<div class="fake-img l-page-outset">
  <p>.l-page-outset</p>
</div>

Occasionally you’ll want to use the full browser width.
For this, use `.l-screen`.
You can also inset the element a little from the edge of the browser by using the inset variant.

<div class="fake-img l-screen">
  <p>.l-screen</p>
</div>
<div class="fake-img l-screen-inset">
  <p>.l-screen-inset</p>
</div>

The final layout is for marginalia, asides, and footnotes.
It does not interrupt the normal flow of `.l-body` sized text except on mobile screen sizes.

<div class="fake-img l-gutter">
  <p>.l-gutter</p>
</div>
***

## Arbitrary $\LaTeX$ (experimental)

In fact, you can write entire blocks of LaTeX using `<latex-js>...</latex-js>` tags.
Below is an example:<d-footnote>If you don't see anything, it means that your browser does not support Shadow DOM.</d-footnote>

<latex-js style="border: 1px dashed #aaa;">
This document will show most of the supported features of \LaTeX.js.

\section{Characters}

It is possible to input any UTF-8 character either directly or by character code
using one of the following:

\begin{itemize}
    \item \texttt{\textbackslash symbol\{"00A9\}}: \symbol{"00A9}
    \item \verb|\char"A9|: \char"A9
    \item \verb|^^A9 or ^^^^00A9|: ^^A9 or ^^^^00A9
\end{itemize}

\bigskip

\noindent
Special characters, like those:
\begin{center}
\$ \& \% \# \_ \{ \} \~{} \^{} \textbackslash % \< \>  \"   % TODO cannot be typeset
\end{center}
%
have to be escaped.

More than 200 symbols are accessible through macros. For instance: 30\,\textcelsius{} is
86\,\textdegree{}F.
</latex-js>

Note that you can easily interleave latex blocks with the standard markdown.

<latex-js style="border: 1px dashed #aaa;">
\section{Environments}

\subsection{Lists: Itemize, Enumerate, and Description}

The \texttt{itemize} environment is suitable for simple lists, the \texttt{enumerate} environment for
enumerated lists, and the \texttt{description} environment for descriptions.

\begin{enumerate}
    \item You can nest the list environments to your taste:
        \begin{itemize}
            \item But it might start to look silly.
            \item[-] With a dash.
        \end{itemize}
    \item Therefore remember: \label{remember}
        \begin{description}
            \item[Stupid] things will not become smart because they are in a list.
            \item[Smart] things, though, can be presented beautifully in a list.
        \end{description}
    \item Technical note: Viewing this in Chrome, however, will show too much vertical space
        at the end of a nested environment (see above). On top of that, margin collapsing for inline-block
        boxes is not allowed. Maybe using \texttt{dl} elements is too complicated for this and a simple nested
        \texttt{div} should be used instead.
\end{enumerate}
%
Lists can be deeply nested:
%
\begin{itemize}
  \item list text, level one
    \begin{itemize}
      \item list text, level two
        \begin{itemize}
          \item list text, level three

            And a new paragraph can be started, too.
            \begin{itemize}
              \item list text, level four
    
                And a new paragraph can be started, too.
                This is the maximum level.
    
              \item list text, level four
            \end{itemize}
    
          \item list text, level three
        \end{itemize}
      \item list text, level two
    \end{itemize}
  \item list text, level one
  \item list text, level one
\end{itemize}

\section{Mathematical Formulae}

Math is typeset using KaTeX. Inline math:
$
f(x) = \int_{-\infty}^\infty \hat f(\xi)\,e^{2 \pi i \xi x} \, d\xi
$
as well as display math is supported:
$$
f(n) = \begin{cases} \frac{n}{2}, & \text{if } n\text{ is even} \\ 3n+1, & \text{if } n\text{ is odd} \end{cases}
$$

</latex-js>

Full $$\LaTeX$$ blocks are supported through [LaTeX JS](https://latex.js.org/){:target="\_blank"} library, which is still under development and supports only limited functionality (which is still pretty cool!) and does not allow fine-grained control of the layout, fonts, etc.

*Note: We do not recommend using using LaTeX JS for writing lecture notes at this stage.*

***

## Print

Finally, you can easily get a PDF or printed version of the notes by simply hitting `ctrl+P` (or `⌘+P` on macOS). -->


## Recap: Control as Inference


<figure id="basic-structure" class="l-body">
    <div class="row">
        <div class="col two">
            <img src="images/recap_control.png" />
        </div>
    </div>
</figure>

In Control setting:

* Initial State $$s_0 \sim p_0(s)$$
* Transition $$s_{t+1} \sim p(s_{t+1} \mid s_t, a_t)$$
* Policy $$a_{t+1} \sim \pi(a_t \mid s_t)$$
* Reward $$r_t \sim r(s_t, a_t)$$


<figure id="basic-structure" class="l-body">
    <div class="row">
        <div class="col two">
            <img src="images/recap_inference.png" />
        </div>
    </div>
</figure>

In Inference setting:

* Initial State $$s_0 \sim p_0(s)$$
* Transition $$s_{t+1} \sim p(s_{t+1} \mid s_t, a_t)$$
* Policy $$a_{t+1} \sim \pi(a_t \mid s_t)$$
* Reward $$r_t \sim r(s_t, a_t)$$
* Optimality $$p(\mathcal{O_t} = 1 \mid s_t, a_t) = exp(r(s_t, a_t))$$


<figure id="basic-structure" class="l-body">
    <div class="row">
        <div class="col two">
            <img src="images/recap_classical.png" />
        </div>
    </div>
</figure>

In the classical deterministic RL setup, we have:

<d-math block>
\begin{aligned} 
& V_\pi(s) := \mathbb{E}_\pi \displaystyle \sum_{k=0}^T \left[\gamma^k r_{t+k+1} \mid s_t = s \right]\\ 
& Q_\pi (s, a) := \mathbb{E}_\pi \displaystyle \sum_{k=0}^T \left[\gamma^k r_{t+k+1} \mid s_t = s, a_t = a \right]\\ 
& \pi_*(a \mid s) := \delta(a = \underset{a}{\text{argmax}} Q_*(s,a))\\
\end{aligned}
</d-math>

Let $$V_t(s_t) = \log \beta_t(s_t), Q_t(s_t, a_t) = \log \beta_t(s_t, a_t), V(s_t) = \log \int exp(Q(s_t, a_t) + \log p(a_t \mid s_t)) da_t$$. 
Denote $$\tau = (s_1,a_1, …, s_T,a_T)$$ as the full trajectory. Denote $p(\tau) = p(\tau \mid \mathcal{O}_{1:T})$. Running inference in this GM allows us to compute:

<d-math block>
\begin{aligned} 
& p(\tau \mid \mathcal{O}_{1:T}) \propto p(s_t) \displaystyle \prod_{t=2}^T p(s_{t+1} \mid s_t, a_t) \times exp(\displaystyle \sum_{t=1}^T r(s_t, a_t)) \\ 
& p(a_t \mid s_t, \mathcal{O}_{1:T}) \propto exp(Q_t(s_t, a_t) - V_t(s_t))​ \\
\end{aligned}
</d-math>

From the perspective of control as inference, we optimize for the following objective:

$$-D_{KL}(\hat{p}(\tau) \mid\mid p(\tau)) = \sum_{t=1}^T \mathbb{E}_{(s_t, a_t) \sim \hat p(s_t, a_t)}\left[r(s_t, a_t)\right] + \mathbb{E}_{s_t \sim \hat{p}(s_t)}\left[\mathcal{H}(\pi(a_t \mid s_t))\right]$$

* For deterministic dynamics, we get this objective directly.
* For stochastic dynamics, we obtain it from the ELBO on the evidence.

## Types of RL Algorithms

The objective of RL learning is to find the optimal parameter s.t. maximize the expected reward.

$$\theta^*=\arg\max_\theta \mathbb{E}_{\tau\sim p(\tau)}\left[\sum_{t=1}^T r(s_t,a_t)\right]$$

* Policy gradients: directly optimize the above stochastic objective
* Value-based: estimate V-function or Q-function of the optimal policy (no explicit policy; the policy is derived from the value function)
* Actor-critic: estimate V-/Q-function under the current policy and use it toimprove the policy (not covered)
* Model-based methods: not covered

## Policy gradients

In policy gradient, we directly optimize the target expected reward w.r.t the policy $\pi_\theta$ itself.

$$\begin{aligned}
J(\theta)&=\mathbb{E}_{\tau \sim p_{\theta}(\tau)}\left[\sum_{t=1}^{T} r\left(s_{t}, a_{t}\right)\right] \approx \frac{1}{N} \sum_{i=1}^{N} \sum_{t=1}^{T} r\left(s_{i, t}, a_{i, t}\right) \\
\nabla_{\theta} J(\theta) &=\nabla_{\theta} \mathbb{E}_{\tau \sim p_{\theta}(\tau)}[r(\tau)]=\int r(\tau) \nabla_{\theta} p_{\theta}(\tau) d \tau=\int r(\tau) p_{\theta}(\tau) \nabla_{\theta} \log p_{\theta}(\tau) d \tau \\ &=\mathbb{E}_{\tau \sim p_{\theta}(\tau)}\left[r(\tau) \nabla_{\theta} \log p_{\theta}(\tau)\right] 
\end{aligned}$$ 

The log-derivative trick is applied to the above equation.
$$
\begin{array}{l}{\nabla_{\theta} \log p_{\theta}(\tau)=\nabla_{\theta}\left[\log p\left(s_{1}\right)+\sum_{t=1}^{T} \log p\left(s_{t+1} | s_{t}, a_{t}\right)+\log \pi_{\theta}\left(a_{t} | s_{t}\right)\right]} \\ {\nabla_{\theta} J(\theta)=\mathbb{E}_{\tau \sim p_{\theta}(\tau)}\left[\left(\sum_{t=1}^{T} \nabla_{\theta} \log \pi_{\theta}\left(a_{t} | s_{t}\right)\right)\left(\sum_{t=1}^{T} r\left(s_{t}, a_{t}\right)\right)\right]}\end{array}$$

The reinforce algorithm:
$$
\begin{array}{l}{\text { 1. sample }\left\{\tau_{i}\right\}_{i=1}^{N} \text { under } \pi_{\theta}\left(a_{t} | s_{t}\right)} \\ {\text { 2. } \hat{J}(\theta)=\sum_{i}\left(\sum_{t} \log \pi_{\theta}\left(a_{i, t} | s_{i, t}\right)\right)\left(\sum_{t} r\left(s_{i, t}, a_{i, t}\right)\right)} \\ {\text { 3. } \theta \leftarrow \theta+\alpha \nabla_{\theta} \hat{J}(\theta)}\end{array}
$$
## Q-Learning

Q-learning does not explicitly optimize the policy $$\pi_\theta$$; it optimize the estimation of the $$V,Q$$ functions. The optimal policy can then be calculated by
$$
\pi^{\prime}\left(a_{t} | s_{t}\right)=\delta\left(a_{t}=\arg \max _{a}\left[Q_{\pi}\left(a, s_{t}\right)\right]\right)
$$
### Policy iteration via dynamic programming:

* Policy iteration
$$\begin{array}{l}{\text { 1. evaluate } Q_{\pi}(s, a)=r(s, a)+\gamma \mathbb{E}_{s^{\prime} \sim p\left(s^{\prime} | s, a\right)}\left[V_{\pi}\left(s^{\prime}\right)\right]} \\ {\text { 2. update } \pi \leftarrow \pi^{\prime}}\end{array}$$

* Policy evaluation
$$V_{\pi}(s) \leftarrow r(s, \pi(s))+\gamma \mathbb{E}_{s^{\prime} \sim p\left(s^{\prime} | s, \pi(s)\right)}\left[V_{\pi}\left(s^{\prime}\right)\right]$$

The approach still involves explicit optimization of $$\pi_\theta$$. We can rewrite the iteration as:

$$\begin{array}{l}{\text { 1. set } Q(s, a) \leftarrow r(s, a)+\gamma \mathbb{E}_{s^{\prime} \sim p\left(s^{\prime} | s, a\right)}\left[V\left(s^{\prime}\right)\right]} \\ {\text { 2. } \operatorname{set} V(s) \leftarrow \max _{a} Q(s, a)}\end{array}$$

### Fitted Q-learning:

If the state space is high-dimensional or infinite, it is not feasible to represent $$Q, V$$ in a tabular form. In this case, we use two parameterized functions $$Q_\phi, V_\phi$$ to denote them. Then, we adopt fitted Q-iteration as stated in [this paper](http://www.jmlr.org/papers/volume6/ernst05a/ernst05a.pdf):

$$\begin{array}{l}{\text { 1. set } y_{i} \leftarrow r\left(s_{i}, a_{i}\right)+\gamma \mathbb{E}_{s^{\prime} \sim p\left(s^{\prime} | s, a\right)}\left[V_{\phi}\left(s_{i}^{\prime}\right)\right]} \\ {\text { 2. } \operatorname{set} \phi \leftarrow \arg \min _{\phi} \sum_{i}\left\|Q_{\phi}\left(s_{i}, a_{i}\right)-y_{i}\right\|^{2}}\end{array}$$



## Soft Policy Gradients
From the perspective of control as inference, we optimize for the following objective:



<d-math block>
\begin{aligned}
J(\theta) &= -D_{KL}(\hat{p}(\tau) || p(\tau)) \\
&= \sum_{t=1}^T \mathbb{E}_{(s_t, a_t) \sim \hat{p(s_t, a_t)}}[r(s_t, a_t)] + \mathbb{E}_{s_t \sim \hat{p}(s_t)}[\mathcal{H}(\pi(a_t|s_t))]\\
&= \sum_{t=1}^T \mathbb{E}_{(s_t, a_t) \sim p_\theta(s_t, a_t)}[r(s_t, a_t) - \log \pi(a_t|s_t)]\\
\end{aligned}​
</d-math>

Now following the policy gradient method, such as REINFORCE, we just need to add a bonus entropy term to the rewards.

## Soft Q-learning
Next, we connect the previous policy gradient to Q-learning. We can rewrite the policy gradient as follows:
<d-math block>
\begin{aligned}
J(\theta) &= -D_{KL}(\hat{p}(\tau) || p(\tau)) \\
&= \sum_{t=1}^T \mathbb{E}_{(s_t, a_t) \sim \hat{p(s_t, a_t)}}[r(s_t, a_t)] + \mathbb{E}_{s_t \sim \hat{p}(s_t)}[\mathcal{H}(\pi(a_t|s_t))]\\
&= \sum_{t=1}^T \mathbb{E}_{(s_t, a_t) \sim p_\theta(s_t, a_t)}[r(s_t, a_t) - \log \pi(a_t|s_t)]\\
\end{aligned}
</d-math>

Note that
<d-math block>
\begin{aligned}
\nabla_\theta \sum_{t=1}^T \mathbb{E}_{(s_t, a_t) \sim p_\theta(s_t, a_t)}[\log \pi(a_t|s_t)] &= \int \nabla_\theta \big[ p(\tau) \sum_{t=1}^T \log \pi(a_t|s_t)\big] d\tau\\
&= \int \nabla_\theta p(\tau)\sum_{t=1}^T \log \pi(a_t|s_t) + p(\tau) \nabla_\theta\sum_{t=1}^T \log \pi(a_t|s_t) d\tau\\
&= \int p(\tau) \nabla_\theta \log p(\tau)\sum_{t=1}^T \log \pi(a_t|s_t) + p(\tau) \nabla_\theta \log p(\tau) d\tau\\
&= \int p(\tau) \nabla_\theta \log p(\tau)\Big [ \sum_{t=1}^T \log \pi(a_t|s_t) + 1 \Big] d\tau.\\
\end{aligned}
</d-math>

Recall from the previous lecture,
$$
\pi(a_t|s_t) = p(a_t | s_t, \mathcal{O}_{1:T}) = \exp(Q_\theta(s_t, a_t) - V_\theta(s_t))
$$

$$
V(s_t) = \log \int \exp(Q(s_t, a_t))da_t = \text{softmax}_{a_t} Q(s_t, a_t).
$$
Now combine these and rearrange the terms, we get:
<d-math block>
\begin{aligned}
\nabla_\theta J(\theta) &= \nabla_\theta \sum_{t=1}^T \mathbb{E}_{(s_t, a_t) \sim p_\theta(s_t, a_t)}[r(s_t, a_t) - \log \pi(a_t|s_t)]\\
&\approx \frac{1}{N}\sum_{i=1}^N\sum_{t=1}^T \nabla_\theta \log\pi_\theta(a_t|s_t) \Big[ r(s_t, a_t) + \big(\sum_{t'=t+1}^T r(s_{t'}, a_{t'}) - \log \pi_\theta(a_{t'}|s_{t'})\big) - \log \pi_\theta (a_t|s_t) - 1 \Big] \\
&= \frac{1}{N}\sum_{i=1}^N\sum_{t=1}^T \Big(\nabla_\theta Q_\theta(s_t, a_t) - \nabla_\theta V_\theta(s_t)\Big) \Big[ r(s_t, a_t) +  Q_\theta(s_{t'}, a_{t'}) - Q_\theta(s_t, a_t) + V(s_t) \Big] \\
&\approx \frac{1}{N}\sum_{i=1}^N\sum_{t=1}^T \nabla_\theta Q_\theta(s_t, a_t) \Big[ r(s_t, a_t) + \text{soft} \max_{a_{t'}}  Q_\theta(s_{t'}, a_{t'}) - Q_\theta(s_t, a_t) \Big] \\
\end{aligned}
</d-math>

Now the soft Q-learning update is very similar to Q-learning:
$$
\theta \gets \theta + \alpha\nabla_\theta Q_\theta(s,a)(r(s,a) + \gamma V(s') - Q_\theta(s,a)),
$$
where
$$
V(s') = \text{soft}\max_{a'} Q_\theta(s', a') = \log \int \exp (Q_\theta(s', a')) da'.
$$
Additionally, we can set the temperature in softmax to control the tradeoff between entropy and rewards.

To summaize, there are a few benefits of soft optimality:
 * Improve exploration and prevent entropy collapse
 * Easier to specialize (finetune) policies for more specific tasks
 * Principled approach to break ties
 * Better robustness (due to wider coverage of states)
 * Can reduce to hard optimality as reward magnitude increases
